In [3]:
#### Necessary Imports
import os
import logging
import inspect 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from torch import nn
import torch
import subprocess
from torch.utils.data import Dataset, DataLoader
import math
import time
import seaborn as sns
import matplotlib as mpl
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import shutil
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score,confusion_matrix
from sklearn.decomposition import PCA
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
import seaborn as sns


import wandb
import random

In [4]:
# pip install wandb

In [5]:
# Validate the parameter for check

# Read parameters from environment variables
epochs_p = int(os.getenv("EPOCHS", 5))
epoch_per_batch_count_p = int(os.getenv("EPOCH_PER_BATCH_COUNT", 2))
weighted_loss_factor_p = float(os.getenv("WEIGHTED_LOSS_FACTOR", 0.1))
print(f"Parameters: epochs={epochs_p}, epoch_per_batch_count={epoch_per_batch_count_p}, weighted_loss_factor={weighted_loss_factor_p}")


Parameters: epochs=5, epoch_per_batch_count=2, weighted_loss_factor=0.1


# Define Constants

In [ ]:
LOG_LEVEL = "INFO" # Adjust this to "DEBUG", "INFO", "WARNING" or "ERROR"

WANDB_API_KEY = "b15ee5c84e51289dd7b5dd11ea38949957d772f9"


activity_id_mapping = {
    0 : "Stand",
    1 : "Sit",
    2 : "Talk-sit",
    3 : "Talk-stand",
    4 : "Stand-sit",
    5 : "Lay",
    6 : "Lay-stand",
    7 : "Pick",
    8 : "Jump",
    9 : "Push-up",
    10 : "Sit-up",
    11 : "Walk",
    12 : "Walk-backward",
    13 : "Walk-circle",
    14 : "Run",
    15 : "Stair-up",
    16 : "Stair-down",
    17 : "Table-tennis"
}

columns = ["activityID", "subjectID", 
               "acc_x", "acc_y", "acc_z", 
               "gyro_x", "gyro_y", "gyro_z", 
               "ori_x", "ori_y", "ori_z"]

EPOCH_BATCH_COUNT = 20
EPOCHS = 5
BATCH_SIZE = 2
IMU_FEATURE_COUNT = 24
CLASSES = 18
LEARNING_RATE = 0.0005 # change learning rate < 0.01

BEST_ACCURACY = 0.0
BEST_LOSS = 1000

MAX_SAVED_MODELS = 6   # Maximum number of models to keep


WEIGHT_LOSS= 0.5

# WEIGHT_CLASSIFIER_LOSS  = 0.9  # >= 8 

# WEIGHT_TRIPLET_LOSS = 0.09  # < 0.15


WEIGHT_NPAIR_LOSS = 0.2
WEIGHT_CLASSIFIER_LOSS  = 1 - WEIGHT_NPAIR_LOSS

SEQ_LENTH = 500
WINDOW_SIZE = 20
NUM_NEGATIVES = 4

# Export logs

In [ ]:
wandb.login(key=WANDB_API_KEY)

In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="har-using-imu-data-npair",
#     name = "Version-17-lr",
    name = "Version-test-Tigger",
#     name = "Interctive-7",
#     notes = "Same as version 53, losss- Triplet cousine",

    # track hyperparameters and run metadata
    config={
        "architecture": "Transformer",
        "dataset": "KU-HAR",
        "epochs": EPOCHS,
        "epoch_batch_count" : EPOCH_BATCH_COUNT,
        "batch_size" : BATCH_SIZE,
        "imu_feature_count" : IMU_FEATURE_COUNT,
        "classes" : CLASSES,
        "learning_rate" : LEARNING_RATE,
        "weight_loss" : WEIGHT_LOSS,
        "WEIGHT_CLASSIFIER_LOSS" : WEIGHT_CLASSIFIER_LOSS,
#         "WEIGHT_TRIPLET_LOSS" : WEIGHT_TRIPLET_LOSS,
        "WEIGHT_NPAIR_LOSS" : WEIGHT_NPAIR_LOSS,
        "SEQ_LENTH" : SEQ_LENTH,
        "WINDOW_SIZE" : WINDOW_SIZE,
        "NUM_NEGATIVES" : NUM_NEGATIVES,
    }
)

# # simulate training
# epochs = 10
# offset = random.random() / 5
# for epoch in range(2, epochs):
#     acc = 1 - 2 ** -epoch - random.random() / epoch - offset
#     loss = 2 ** -epoch + random.random() / epoch + offset

#     # log metrics to wandb
#     wandb.log({"acc": acc, "loss": loss})

# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

# Genarate logs

In [ ]:
# Map log level strings to logging constants
log_levels = {
    "DEBUG": logging.DEBUG,
    "INFO": logging.INFO,
    "WARNING": logging.WARNING,
    "ERROR": logging.ERROR
}
set_log_level = log_levels.get(LOG_LEVEL, logging.INFO)  # Default to INFO if an unrecognized level is given

# Configure the logging format and level
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=set_log_level,
    datefmt="%Y-%m-%d %H:%M:%S"
)

def log_message(level, message, block=None, log_title="", caller_frame=None):
    """
    Logs a message with a specific logging level and additional details.
    
    Args:
        level (str): Logging level ('DEBUG', 'INFO', 'WARNING', 'ERROR').
        message (str): The message to log.
        block (str, optional): Additional block/section name for context.
        log_title (str): Title to specify log type.
        caller_frame (frame, optional): Frame object of the calling function.
    """
    # Get the calling function's details for context
    line_number = caller_frame.f_lineno if caller_frame else "N/A"
    function_name = caller_frame.f_code.co_name if caller_frame else "N/A"

    # Format log title and line number to a fixed width
    formatted_title = log_title.ljust(7)  # Pad log title to 7 characters
    formatted_line = f"Line {line_number}".ljust(8)  # Pad line info to 8 characters

    # Format the log with extra details
    # log_msg = f"{formatted_title} | {formatted_line} | {function_name} | {message}"
    log_msg = f"{formatted_title} | {formatted_line} | {message}"
    if block:
        log_msg += f" - Block: {block}"

    # Check if the log level should print based on the configured log level
    should_print = log_levels[level.upper()] >= set_log_level

    if should_print:
        print(log_msg)  # Print the message if it meets or exceeds the log level

    # Log the message based on the specified level
    if level.upper() == "DEBUG":
        logging.debug(log_msg)
    elif level.upper() == "INFO":
        logging.info(log_msg)
    elif level.upper() == "WARNING":
        logging.warning(log_msg)
    elif level.upper() == "ERROR":
        logging.error(log_msg)
    else:
        logging.info("Unknown log level specified.")

# Shortcut functions for different levels
def print_log(message, block=None):
    caller_frame = inspect.currentframe().f_back
    log_message("INFO", message, block, log_title="INFO", caller_frame=caller_frame)

def debug_log(message, block=None):
    caller_frame = inspect.currentframe().f_back
    log_message("DEBUG", message, block, log_title="DEBUG", caller_frame=caller_frame)

def warn_log(message, block=None):
    caller_frame = inspect.currentframe().f_back
    log_message("WARNING", message, block, log_title="WARNING", caller_frame=caller_frame)

def error_log(message, block=None):
    caller_frame = inspect.currentframe().f_back
    log_message("ERROR", message, block, log_title="ERROR", caller_frame=caller_frame)

In [ ]:
# The example usage of logs
print_log("Training started", block="Training Phase")
debug_log("Loaded 500 records", block="Data Loading")
warn_log("Missing values detected", block="Data Validation")
error_log("Failed to save model", block="Model Saving")

# Clear Working Directory

In [ ]:
import os
import shutil


def clear_working_directory():
    # Define the directory to clear
    directory_to_clear = '/kaggle/working/'

    # Iterate through all files and directories in the specified directory
    for filename in os.listdir(directory_to_clear):
        file_path = os.path.join(directory_to_clear, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove the file or symbolic link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove the directory and its contents
            print_log(f"Removed: {file_path}")  # Optional: Print the removed file or directory
        except Exception as e:
            error_log(f"Failed to remove {file_path}. Reason: {e}")


In [ ]:
clear_working_directory()

# Preprocessing Data

In [ ]:
#Reading data:

df = pd.read_csv("/kaggle/input/3.Time_domain_subsamples/KU-HAR_time_domain_subsamples_20750x300.csv",header=None)
dff = df.values
signals = dff[:, 0: 1800] #These are the time-domian subsamples (signals) 
signals = np.array(signals, dtype=np.float32)
labels = dff[:, 1800] #These are their associated class labels (signals)

print_log(f"signals shape: {signals.shape} and labels shape: {labels.shape}")


In [ ]:
# Reshape signals into 300 samples per axis for each sensor type (acc, gyro, orientation)
# For accelerometer data (X, Y, Z axis)
acc_x = signals[:, :300]
acc_y = signals[:, 300:600]
acc_z = signals[:, 600:900]
gyro_x = signals[:, 900:1200]
gyro_y = signals[:, 1200:1500]
gyro_z = signals[:, 1500:1800]

# Create a list to hold all rows of data
data_rows = []

# Loop over each activity ID and create rows for each sample and axis
for i in range(len(labels)):
    activity_id = labels[i]
    
    # For each of the 300 samples, create a row with each sensor axis value
    for sample in range(300):
        data_rows.append({"activityID": int(activity_id), 
                          "acc_x": acc_x[i, sample],
                         "acc_y" : acc_y[i, sample],
                         "acc_z" : acc_z[i, sample],
                         "gyro_x" : gyro_x[i, sample],
                         "gyro_y" : gyro_y[i, sample],
                         "gyro_z" : gyro_z[i, sample]})

# Create a DataFrame from the expanded data
new_df = pd.DataFrame(data_rows)
print_log("The basic new dataframe is genarated")

# **Standardize the dataframe** 
Standardize the record count across all activities for each subject.

In [ ]:
# activity_counts = new_df.groupby('activityID').size()

# # Print the counts for each activityID
# print(activity_counts)

In [ ]:
# Count the number of records for each activityID
activity_counts = new_df.groupby('activityID').size()

# Find the minimum count of records
min_records = activity_counts.min()

# Print the minimum count and the corresponding activityID(s)
min_activity_ids = activity_counts[activity_counts == min_records].index.tolist()

print_log(f"Minimum number of records: {min_records}")
print_log(f"Activity IDs with minimum records: {min_activity_ids}")


In [ ]:
# Count the number of records for each activityID
activity_counts = new_df.groupby('activityID').size()

# Find the minimum count of records
min_records = activity_counts.min()

# Initialize a list to store the resampled DataFrames
equalized_dfs = []

# Iterate through each activityID
for activity_id in activity_counts.index:
    # Filter the DataFrame for the current activityID
    activity_data = new_df[new_df['activityID'] == activity_id]
    
    # Resample the data to the minimum count with replacement
    resampled_data = activity_data.sample(n=min_records, replace=True, random_state=42)
    
    # Append the resampled DataFrame to the list
    equalized_dfs.append(resampled_data)

# Concatenate all resampled DataFrames into one
equalized_df = pd.concat(equalized_dfs, ignore_index=True)
print_log("The dataframe is Standardized")

# Show the final equalized DataFrame
print(equalized_df.groupby('activityID').size())  # Check the counts per activityID

# Reassign the dataframe
new_df = equalized_df
new_df

# Visualize

In [ ]:
def visualize_activity_data(activity_id, df):
    # Filter data for the specific activity ID
    activity_data = df[df['activityID'] == activity_id]
    
    # Ensure there are enough samples for plotting (300 samples)
    if len(activity_data) < 300:
        warn_log(f"Not enough samples for activity ID {activity_id}")
        return
    
    # Extract sensor data for the first 300 samples for each axis
    acc_x = activity_data['acc_x'].values[:300]
    acc_y = activity_data['acc_y'].values[:300]
    acc_z = activity_data['acc_z'].values[:300]
    gyro_x = activity_data['gyro_x'].values[:300]
    gyro_y = activity_data['gyro_y'].values[:300]
    gyro_z = activity_data['gyro_z'].values[:300]
    
    # Generate time array (assuming each sample is 0.01 seconds apart)
    time = np.linspace(0.01, 3, 300)
    
    # Create figure for acceleration
    plt.figure(figsize=(10, 6))
    
    # Plot accelerometer X, Y, Z axis on the same plot
    plt.plot(time, acc_x, color='b', label='Accelerometer X')
    plt.plot(time, acc_y, color='g', label='Accelerometer Y')
    plt.plot(time, acc_z, color='r', label='Accelerometer Z')
    
    plt.title('Accelerometer Data (X, Y, Z)')
    plt.xlabel('time (s)')
    plt.ylabel('Acceleration (m/s^2)')
    plt.grid(True)
    plt.legend()
    print_log("Accelerometer plot is created")
    
    # Show the first plot
    plt.tight_layout()
    plt.show()
    
    # Create figure for gyroscope
    plt.figure(figsize=(10, 6))
    
    # Plot gyroscope X, Y, Z axis on the same plot
    plt.plot(time, gyro_x, color='b', label='Gyroscope X')
    plt.plot(time, gyro_y, color='g', label='Gyroscope Y')
    plt.plot(time, gyro_z, color='r', label='Gyroscope Z')
    
    plt.title('Gyroscope Data (X, Y, Z)')
    plt.xlabel('time (s)')
    plt.ylabel('Angular rotation (rad/s)')
    plt.grid(True)
    plt.legend()
    print_log("Gyroscope plot is created")
    
    # Show the second plot
    plt.tight_layout()
    plt.show()


In [ ]:
visualize_activity_data(activity_id=9, df=new_df)

# Transform the data

In [ ]:
# Function to create a DataFrame with original values, derivatives, and Fourier transforms
def create_transformed_df(df):
    transformed_data = {}

    # Include original values for each column, including activityID
    transformed_data['activityID'] = df['activityID']
    for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']:
        transformed_data[col] = df[col]  # Add original data

        # Compute first derivative
        transformed_data[f'{col}_fd'] = np.gradient(df[col])
        
        # Compute second derivative
        transformed_data[f'{col}_sd'] = np.gradient(transformed_data[f'{col}_fd'])
        
        # Compute Fourier Transform (absolute values to keep real magnitudes)
        transformed_data[f'{col}_fourier'] = np.abs(np.fft.fft(df[col]))

    # Create the final DataFrame
    transformed_df = pd.DataFrame(transformed_data)
    print_log("The first and secind derivatives, and Fourier transforms values of sensor data added in the dataframe")

    return transformed_df


In [ ]:
def assign_subject_ids(df):
    # Get unique activity IDs in the dataset
    activity_ids = df['activityID'].unique()
    
    # Initialize list to hold subject IDs
    subject_ids = []
    
    # Loop over each activity ID
    for activity_id in activity_ids:
        # Get subset for the specific activity
        activity_df = df[df['activityID'] == activity_id]
        
        # Number of records for the activity
        n_records = len(activity_df)
        
        # Use round-robin assignment of subject IDs
        subject_id_repeated = np.tile(np.arange(1, 11), n_records // 10 + 1)[:n_records]
        
        # Append these subject IDs to the list
        subject_ids.extend(subject_id_repeated)
    
    # Add subjectID to the DataFrame
    df['subjectID'] = subject_ids
    print_log("subjectID is assigned for every record")
    return df


def assign_subject_ids_random(df):
    # Get unique activity IDs in the dataset
    activity_ids = df['activityID'].unique()
    
    # Initialize list to hold subject IDs
    subject_ids = []
    
    # Loop over each activity ID
    for activity_id in activity_ids:
        # Get subset for the specific activity
        activity_df = df[df['activityID'] == activity_id]
        
        # Number of records for the activity
        n_records = len(activity_df)
        
        # Randomly assign subject IDs from 1 to 10
        subject_id_random = np.random.choice(np.arange(1, 11), n_records, replace=True)
        
        # Append these subject IDs to the list
        subject_ids.extend(subject_id_random)
    
    # Add subjectID to the DataFrame
    df['subjectID'] = subject_ids
    print_log("Random subjectID is assigned for every record")
    return df



In [ ]:
# cretae transformed df it include first and second derivative and fourior, Assign the subject IDs from 0 to 10
transformed_df = create_transformed_df(new_df)
final_df = assign_subject_ids(transformed_df)

In [ ]:
activity_counts = final_df.groupby('activityID').size()

# Print the counts for each activityID
print_log(activity_counts)

In [ ]:
def split_data(df, train_ratio=0.7, test_ratio=0.2, validation_ratio=0.1):
    # Validate that the ratios sum to 1
    if not (train_ratio + test_ratio + validation_ratio != 1):
        error_log("Train : Test: Validation => Ratios must sum to 1.")
        raise ValueError("Ratios must sum to 1.")
    
    # Initialize empty DataFrames for train, test, and validation
    train_df = pd.DataFrame(columns=df.columns)
    test_df = pd.DataFrame(columns=df.columns)
    validation_df = pd.DataFrame(columns=df.columns)

    # Iterate over each group of (activityID, subjectID)
    for (activity_id, subject_id), group in df.groupby(['activityID', 'subjectID']):
        n = len(group)
        
        if n == 0:
            warn_log(f"The group of activity_id : {activity_id} and subject_id : {subject_id} is empty")
            continue  # Skip empty groups
        
        # Shuffle the group for randomness (if needed)
        group = group.sample(frac=1).reset_index(drop=True)

        # Calculate indices for splitting
        train_end = int(train_ratio * n)
        test_end = train_end + int(test_ratio * n)

        # Split into train, test, validation based on indices
        train_data = group.iloc[:train_end]
        test_data = group.iloc[train_end:test_end]
        validation_data = group.iloc[test_end:]

        # Concatenate only if the DataFrames are not empty
        train_df = pd.concat([train_df, train_data], ignore_index=True)
        test_df = pd.concat([test_df, test_data], ignore_index=True)
        validation_df = pd.concat([validation_df, validation_data], ignore_index=True)

    print_log("Data is split into three parts: train, test, and validation.")

    return train_df, test_df, validation_df


In [ ]:
train_df, test_df, validation_df = split_data(final_df)

print_log(f"Train data shape : {train_df.shape}")
print_log(f"Test data shape: {test_df.shape}")
print_log(f"Validation data shape: {validation_df.shape}")

# # Define the file paths for saving the CSV files
# train_csv_path = '/kaggle/working/train_data.csv'
# test_csv_path = '/kaggle/working/test_data.csv'
# validation_csv_path = '/kaggle/working/validation_data.csv'

# # Save the DataFrames to CSV files
# train_df.to_csv(train_csv_path, index=False)
# print_log("train_data.csv file is created")
# test_df.to_csv(test_csv_path, index=False)
# print_log("test_data.csv file is created")
# validation_df.to_csv(validation_csv_path, index=False)
# print_log("validation_data.csv file is created")

In [ ]:
# train_df.groupby('activityID').size()

# Visualizing Activity Distribution and Understanding Variability

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame from your groupby result
activity_distribution = train_df.groupby(['activityID', 'subjectID']).size().reset_index(name='count')

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(data=activity_distribution, x='activityID', y='count', hue='subjectID')
plt.title('Activity Distribution Across Subjects')
plt.xlabel('Activity ID')
plt.ylabel('Count')
plt.legend(title='Subject ID')
plt.show()

In [ ]:
activity_variability = train_df.groupby('activityID')['subjectID'].value_counts().unstack()
variability_std = activity_variability.std(axis=1)
print_log(f" activity variability : {variability_std} ")


In [ ]:
def generate_sequence_list(df, sequence_length=500, overlap=20):
    # Initialize the main list to hold all sequences for all activities
    imu_data_sequence = []

    # Group by activityID to handle each activity separately
    grouped_by_activity = df.groupby('activityID')

    # Iterate over each activity group
    for activity_id, activity_group in grouped_by_activity:
        # Initialize a list for this activity
        activity_data_sequence = []

        # Group by subjectID within the activity group
        grouped_by_subject = activity_group.groupby('subjectID')
        
        # Iterate over each subject group within the activity
        for subject_id, subject_group in grouped_by_subject:
            # Extract the data values excluding activityID and subjectID
            data_values = subject_group.drop(columns=['activityID', 'subjectID']).values
            
            # Calculate the number of sequences
            num_samples = len(data_values)
            num_sequences = (num_samples - sequence_length) // overlap + 1
            
            # Create a list for this subject's sequences
            subject_sequences = []
            
            # Generate sequences for this subject
            for i in range(num_sequences):
                sequence_start = i * overlap
                sequence_end = sequence_start + sequence_length
                if sequence_end <= num_samples:
                    # Append the sequence to the subject's sequence list
                    subject_sequences.append(data_values[sequence_start:sequence_end])
            
            # Add the subject sequences to the activity's list
            activity_data_sequence.append(subject_sequences)
            debug_log(f"The sequences are generated for subjectID: {subject_id}")
        
        # After processing all subjects, print activity completion
        print_log(f"The sequences are generated for activityID: {activity_id}")

        # Add the activity data sequence to the main list
        imu_data_sequence.append(activity_data_sequence)

    return imu_data_sequence


In [ ]:
# Generate sequences for each DataFrame
imu_data_sequence_train = generate_sequence_list(train_df, SEQ_LENTH, WINDOW_SIZE)
imu_data_sequence_test = generate_sequence_list(test_df, SEQ_LENTH, WINDOW_SIZE)
imu_data_sequence_validation = generate_sequence_list(validation_df, SEQ_LENTH, WINDOW_SIZE)

In [ ]:
len(imu_data_sequence_train[1][3])

# Store data as a pickle

In [ ]:
# # # Store the imu data as a pickle
# pickle_imu_data_directory = "/kaggle/working/pickle_imu_data"

# # # Create the directory if it doesn't exist
# os.makedirs(pickle_imu_data_directory, exist_ok=True)

# # # File paths for pickle files
# # train_pickle_path = os.path.join(pickle_imu_data_directory, 'imu_data_sequence_train.pkl')
# test_pickle_path = os.path.join(pickle_imu_data_directory, 'imu_data_sequence_test.pkl')
# # validation_pickle_path = os.path.join(pickle_imu_data_directory, 'imu_data_sequence_validation.pkl')

# # # Save the training data sequence to a pickle file
# # with open(train_pickle_path, 'wb') as train_file:
# #     pickle.dump(imu_data_sequence_train[0:8], train_file)
# # print_log("Train Pickle file has been saved successfully")

# # # Save the testing data sequence to a pickle file
# with open(test_pickle_path, 'wb') as test_file:
#     pickle.dump(imu_data_sequence_test[0:8], test_file)
# print_log("Test Pickle file has been saved successfully")

# # # Save the validation data sequence to a pickle file
# # with open(validation_pickle_path, 'wb') as validation_file:
# #     pickle.dump(imu_data_sequence_validation[0:8], validation_file)
# # print_log("Validation Pickle file has been saved successfully")

# # print("Pickle files have been saved successfully.")



# Model

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, k, d_model, seq_len):
        super().__init__()
        
        self.embedding = nn.Parameter(torch.zeros([k, d_model], dtype=torch.float), requires_grad=True)
        nn.init.xavier_uniform_(self.embedding, gain=1)
        self.positions = torch.tensor([i for i in range(seq_len)], requires_grad=False).unsqueeze(1).repeat(1, k)
        s = 0.0
        interval = seq_len / k
        mu = []
        for _ in range(k):
            mu.append(nn.Parameter(torch.tensor(s, dtype=torch.float), requires_grad=True))
            s = s + interval
        self.mu = nn.Parameter(torch.tensor(mu, dtype=torch.float).unsqueeze(0), requires_grad=True)
        self.sigma = nn.Parameter(torch.tensor([torch.tensor([50.0], dtype=torch.float, requires_grad=True) for _ in range(k)]).unsqueeze(0))
        
    def normal_pdf(self, pos, mu, sigma):
        a = pos - mu
        log_p = -1*torch.mul(a, a)/(2*(sigma**2)) - torch.log(sigma)
        return torch.nn.functional.softmax(log_p, dim=1)

    def forward(self, inputs):
        pdfs = self.normal_pdf(self.positions, self.mu, self.sigma)
        pos_enc = torch.matmul(pdfs, self.embedding)
        
        return inputs + pos_enc.unsqueeze(0).repeat(inputs.size(0), 1, 1)

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, _heads, dropout, seq_len):
        super(TransformerEncoderLayer, self).__init__()
        
        self.attention = nn.MultiheadAttention(d_model, heads, batch_first=True)
        self._attention = nn.MultiheadAttention(seq_len, _heads, batch_first=True)
        
        self.attn_norm = nn.LayerNorm(d_model)
        
        self.cnn_units = 1
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1, self.cnn_units, (1, 1)),
            nn.BatchNorm2d(self.cnn_units),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Conv2d(self.cnn_units, self.cnn_units, (3, 3), padding=1),
            nn.BatchNorm2d(self.cnn_units),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Conv2d(self.cnn_units, 1, (5, 5), padding=2),
            nn.BatchNorm2d(1),
            nn.Dropout(dropout),
            nn.ReLU()
        )
        
        self.final_norm = nn.LayerNorm(d_model)

    def forward(self, src, src_mask=None):
        src = self.attn_norm(src + self.attention(src, src, src)[0] + self._attention(src.transpose(-1, -2), src.transpose(-1, -2), src.transpose(-1, -2))[0].transpose(-1, -2))
        
        src = self.final_norm(src + self.cnn(src.unsqueeze(dim=1)).squeeze(dim=1))
            
        return src

class TransformerEncoder(nn.Module):
    def __init__(self, d_model, heads, _heads, seq_len, num_layer=2, dropout=0.1):
        super(TransformerEncoder, self).__init__()

        self.layers = nn.ModuleList()
        for i in range(num_layer):
            self.layers.append(TransformerEncoderLayer(d_model, heads, _heads, dropout, seq_len))

    def forward(self, src):
        for layer in self.layers:
            src = layer(src)

        return src

class Transformer(nn.Module):
    def __init__(self, num_layer, d_model, k, heads, _heads, seq_len, trg_len, dropout):
        super(Transformer, self).__init__()

        self.pos_encoding = PositionalEncoding(k, d_model, seq_len)

        self.encoder = TransformerEncoder(d_model, heads, _heads, seq_len, num_layer, dropout)

    def forward(self, inputs):
        encoded_inputs = self.pos_encoding(inputs)

        return self.encoder(encoded_inputs)

class Model(nn.Module):
    def __init__(self, feature_count, l, trg_len, num_classes):
        super(Model, self).__init__()
        
        self.imu_transformer = Transformer(5, feature_count, 100, 4, 4, l, trg_len, 0.1)
        
        self.linear_imu = nn.Sequential(
            nn.Linear(feature_count*l, (feature_count*l)//2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear((feature_count*l)//2, trg_len),
            nn.ReLU()
        )
        
        # Batch normalization and dropout layers
        self.batch_norm = nn.BatchNorm1d(trg_len)
        self.dropout = nn.Dropout(0.5)
        
        # Classifier layer
        self.classifier = nn.Linear(trg_len, num_classes)

    def forward(self, inputs):
        
        embedding = self.linear_imu(torch.flatten(self.imu_transformer(inputs), start_dim=1, end_dim=2))
        
        # Apply batch normalization
        embedding = self.batch_norm(embedding)
        
        # Apply dropout
        embedding = self.dropout(embedding)
        
        # Get class scores
        class_scores = self.classifier(embedding)
        
        return class_scores, embedding

# If you have GPU

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
batch_size=BATCH_SIZE
epoch_batch_count=EPOCH_BATCH_COUNT
imu_l=SEQ_LENTH    #sequence length
imu_feature_count=IMU_FEATURE_COUNT
trg_len=128 # this will be the size of feature embedding
classes=CLASSES

In [ ]:
best_model_save_path = '/kaggle/working/best_models'
checkpoint_save_path = '/kaggle/working/checkpoints'

subprocess.run(f"mkdir {best_model_save_path}", shell=True)
subprocess.run(f"mkdir {checkpoint_save_path}", shell=True)

# Train Dataset

In [ ]:
# class TrainDataset(Dataset):
#     def __init__(self, training_data, batch_size, epoch_batch_count):
#         self.training_data = training_data
#         self.batch_size = batch_size
#         self.epoch_batch_count = epoch_batch_count

#     def __len__(self):
#         return self.batch_size * self.epoch_batch_count

#     def __getitem__(self, idx):
#         while True:
#             try:
#                 genuine_user_idx = np.random.randint(0, len(self.training_data))
#                 imposter_user_idx = np.random.randint(0, len(self.training_data))
                
#                 # Ensure imposter_user_idx is different from genuine_user_idx
#                 while imposter_user_idx == genuine_user_idx:
#                     imposter_user_idx = np.random.randint(0, len(self.training_data))
                
#                 # Validate the lengths of genuine_user and imposter_user data
#                 if len(self.training_data[genuine_user_idx]) == 0 or len(self.training_data[imposter_user_idx]) == 0:
#                     raise ValueError("Empty user data detected.")
                
#                 genuine_sess_1 = np.random.randint(0, len(self.training_data[genuine_user_idx]))
#                 genuine_sess_2 = np.random.randint(0, len(self.training_data[genuine_user_idx]))
                
#                 # Ensure genuine_sess_2 is different from genuine_sess_1
#                 while genuine_sess_2 == genuine_sess_1:
#                     genuine_sess_2 = np.random.randint(0, len(self.training_data[genuine_user_idx]))
                
#                 # Validate the lengths of genuine_sess_1 and genuine_sess_2 data
#                 if len(self.training_data[genuine_user_idx][genuine_sess_1]) == 0 or len(self.training_data[genuine_user_idx][genuine_sess_2]) == 0:
#                     raise ValueError("Empty session data detected.")
                
#                 imposter_sess = np.random.randint(0, len(self.training_data[imposter_user_idx]))
                
#                 # Validate the length of imposter_sess data
#                 if len(self.training_data[imposter_user_idx][imposter_sess]) == 0:
#                     raise ValueError("Empty imposter session data detected.")
                
#                 genuine_seq_1 = np.random.randint(0, len(self.training_data[genuine_user_idx][genuine_sess_1]))
#                 genuine_seq_2 = np.random.randint(0, len(self.training_data[genuine_user_idx][genuine_sess_2]))
#                 imposter_seq = np.random.randint(0, len(self.training_data[imposter_user_idx][imposter_sess]))
# #                 debug_log(f"{genuine_user_idx}, {genuine_sess_1}, {genuine_sess_2}, {imposter_user_idx}, {imposter_sess}")
#                 anchor = self.training_data[genuine_user_idx][genuine_sess_1][genuine_seq_1]
#                 positive = self.training_data[genuine_user_idx][genuine_sess_2][genuine_seq_2]
#                 negative = self.training_data[imposter_user_idx][imposter_sess][imposter_seq]

#                 return anchor, positive, negative, genuine_user_idx, imposter_user_idx
            
#             except ValueError as e:
#                 error_log(f"Encountered ValueError: {str(e)}. Retrying with new indices.")

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, training_data, batch_size, epoch_batch_count, num_negatives):
        self.training_data = training_data
        self.batch_size = batch_size
        self.epoch_batch_count = epoch_batch_count
        self.num_negatives = num_negatives

    def __len__(self):
        return self.batch_size * self.epoch_batch_count

    def __getitem__(self, idx):
        while True:
            try:
                genuine_user_idx = np.random.randint(0, len(self.training_data))
                imposter_user_idxs = []
                # Validate the lengths of genuine_user and imposter_user data
                if len(self.training_data[genuine_user_idx]) == 0:
                    raise ValueError("Empty user data detected.")
                
                for i in range(0, self.num_negatives):
                    imposter_user_idx =  np.random.randint(0, len(self.training_data))
                
                    # Ensure imposter_user_idx is different from genuine_user_idx and diffrent form other imposter_user_idx
                    while imposter_user_idx == genuine_user_idx or imposter_user_idx in imposter_user_idxs:
                        imposter_user_idx = np.random.randint(0, len(self.training_data))
                
                    # Validate the lengths of genuine_user and imposter_user data
                    if len(self.training_data[imposter_user_idx]) == 0:
                        raise ValueError("Empty user data detected.")
                    
                    imposter_user_idxs.append(imposter_user_idx)
                
                
                genuine_sess_1 = np.random.randint(0, len(self.training_data[genuine_user_idx]))
                genuine_sess_2 = np.random.randint(0, len(self.training_data[genuine_user_idx]))
                
                # Ensure genuine_sess_2 is different from genuine_sess_1
                while genuine_sess_2 == genuine_sess_1:
                    genuine_sess_2 = np.random.randint(0, len(self.training_data[genuine_user_idx]))
                
                # Validate the lengths of genuine_sess_1 and genuine_sess_2 data
                if len(self.training_data[genuine_user_idx][genuine_sess_1]) == 0 or len(self.training_data[genuine_user_idx][genuine_sess_2]) == 0:
                    raise ValueError("Empty session data detected.")
                
                
                genuine_seq_1 = np.random.randint(0, len(self.training_data[genuine_user_idx][genuine_sess_1]))
                genuine_seq_2 = np.random.randint(0, len(self.training_data[genuine_user_idx][genuine_sess_2]))
                
                anchor = self.training_data[genuine_user_idx][genuine_sess_1][genuine_seq_1]
                positive = self.training_data[genuine_user_idx][genuine_sess_2][genuine_seq_2]
                negatives = []
                
                for i in imposter_user_idxs:
                    imposter_sess = np.random.randint(0, len(self.training_data[imposter_user_idx]))
                
                    # Validate the length of imposter_sess data
                    if len(self.training_data[imposter_user_idx][imposter_sess]) == 0:
                        raise ValueError("Empty imposter session data detected.")
                
                
                    imposter_seq = np.random.randint(0, len(self.training_data[imposter_user_idx][imposter_sess]))
                    negative = self.training_data[imposter_user_idx][imposter_sess][imposter_seq]
                    negatives.append(negative)

                return anchor, positive, negatives, genuine_user_idx, imposter_user_idxs
            
            except ValueError as e:
                error_log(f"Encountered ValueError: {str(e)}. Retrying with new indices.")

# Loss

In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(dim=1).sqrt()
    
    def calc_cosine(self, x1, x2):
        dot_product_sum = (x1*x2).sum(dim=1)
        norm_multiply = (x1.pow(2).sum(dim=1).sqrt()) * (x2.pow(2).sum(dim=1).sqrt())
        return dot_product_sum / norm_multiply
    
    def calc_manhattan(self, x1, x2):
        return (x1-x2).abs().sum(dim=1)
    
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

In [ ]:
# import torch
# import torch.nn as nn

# class NPairLoss(nn.Module):
#     def __init__(self):
#         super(NPairLoss, self).__init__()

#     def calc_cosine(self, x1, x2):
#         dot_product_sum = (x1 * x2).sum(dim=2)  # Change to dim=2 since x2 is now [batch_size, num_negatives, embedding_dim]
#         norm_multiply = (x1.pow(2).sum(dim=2).sqrt()) * (x2.pow(2).sum(dim=2).sqrt())
#         return dot_product_sum / norm_multiply

#     def forward(self, anchor, positive, negatives):
#         # Calculate cosine similarities for the positive pair
#         pos_sim = self.calc_cosine(anchor.unsqueeze(1), positive.unsqueeze(1))  # Shape: (8, 1)
        
#         # Calculate cosine similarities for each negative
#         neg_sim = self.calc_cosine(anchor.unsqueeze(1), negatives.permute(1, 0, 2))  # Shape: (8, 4)

#         # Exponential of the similarities
#         pos_exp = torch.exp(pos_sim.squeeze(1))  # Shape: (8,)
#         neg_exp = torch.exp(neg_sim)  # Shape: (8, 4)

#         # Compute loss
#         numerator = pos_exp  # Shape: (8,)
#         denominator = pos_exp.unsqueeze(1) + neg_exp.sum(dim=1)  # Shape: (8, 1) + (8, 4) -> (8, 4)

#         # Calculate loss
#         loss = -torch.log(numerator / denominator)
#         return loss.mean()  # Average loss over the batch

# # Example usage:
# batch_size = 8
# embedding_dim = 128
# num_negatives = 4

# # Example embeddings
# anchor_features = torch.randn(batch_size, embedding_dim)  # Shape: (8, 128)
# positive_features = torch.randn(batch_size, embedding_dim)  # Shape: (8, 128)
# negative_features = torch.randn(num_negatives, batch_size, embedding_dim)  # Shape: (4, 8, 128)

# # Debug prints to check sizes
# print(f"Anchor features shape: {anchor_features.shape}")
# print(f"Positive features shape: {positive_features.shape}")
# print(f"Negative features shape: {negative_features.shape}")
        

# # Initialize loss function
# loss_fn = NPairLoss()
# loss = loss_fn(anchor_features, positive_features, negative_features)

# print("Loss:", loss.item())


In [ ]:
import torch
import torch.nn as nn

class NPairLoss(nn.Module):
    def __init__(self):
        super(NPairLoss, self).__init__()

    def calc_cosine(self, x1, x2):
        dot_product_sum = (x1 * x2).sum(dim=2)  # Change to dim=2 since x2 is now [batch_size, num_negatives, embedding_dim]
        norm_multiply = (x1.pow(2).sum(dim=2).sqrt()) * (x2.pow(2).sum(dim=2).sqrt())
        return dot_product_sum / norm_multiply

    def forward(self, anchor, positive, negatives):
        # Calculate cosine similarities for the positive pair
        pos_sim = self.calc_cosine(anchor.unsqueeze(1), positive.unsqueeze(1))  # Shape: (8, 1)
        
        # Calculate cosine similarities for each negative
        neg_sim = self.calc_cosine(anchor.unsqueeze(1), negatives.permute(1, 0, 2))  # Shape: (8, 4)

        # Exponential of the similarities
        pos_exp = torch.exp(pos_sim.squeeze(1))  # Shape: (8,)
        neg_exp = torch.exp(neg_sim)  # Shape: (8, 4)

        # Compute loss
        numerator = pos_exp  # Shape: (8,)
        denominator = pos_exp.unsqueeze(1) + neg_exp.sum(dim=1)  # Shape: (8, 1) + (8, 4) -> (8, 4)

        # Calculate loss
        loss = -torch.log(numerator / denominator)
        return loss.mean()  # Average loss over the batch

# Set the classes

In [ ]:
train_data = imu_data_sequence_train[0:CLASSES]
test_data = imu_data_sequence_test[0:CLASSES]
validation_data = imu_data_sequence_validation[0:CLASSES]

In [ ]:
# len(train_data[17][9][88])
# train_data[17][9][88][999]

# Using only some activity classes for improve our model accuracy

In [ ]:
# input_classes = [0,1,2,4, 6, 7]
# train_data = [imu_data_sequence_train[i] for i in input_classes] 
# test_data = [imu_data_sequence_test[i] for i in input_classes]  
# validation_data = [imu_data_sequence_validation[i] for i in input_classes] 

In [ ]:
# dataset = TrainDataset(train_data, batch_size, epoch_batch_count)
# dataloader = DataLoader(dataset, batch_size=batch_size)
num_negatives = NUM_NEGATIVES  # Number of negatives to sample for each anchor

# Instantiate dataset and dataloader
train_dataset = TrainDataset(train_data, batch_size, epoch_batch_count, num_negatives)

dataloader = DataLoader(train_dataset, batch_size=batch_size)


model = Model(imu_feature_count, imu_l, trg_len,classes)


In [ ]:
loss_fn = NPairLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE,weight_decay=1e-5) # change the learning rate
# optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
g_eer = math.inf
init_epoch = 0
epochs=EPOCHS #increase the epochs

In [ ]:
plt.style.use('seaborn-v0_8-bright')
plt.rcParams['axes.facecolor'] = 'white'
mpl.rcParams.update({"axes.grid" : True, "grid.color": "black"})
mpl.rc('axes',edgecolor='black')
mpl.rcParams.update({'font.size': 13})

# Test Dataset

In [ ]:
# class TestDataset(Dataset):
#     def __init__(self, eval_data):
#         self.eval_data = eval_data
#         self.num_sessions = len(self.eval_data[0])
#         self.num_seqs = len(self.eval_data[0][0])

#     def __len__(self):
#         return math.ceil(len(self.eval_data) * self.num_sessions * self.num_seqs)

#     def __getitem__(self, idx):
#         t_session = idx // self.num_seqs
#         user_idx = t_session // self.num_sessions
#         session_idx = t_session % self.num_sessions
#         seq_idx = idx % self.num_seqs
        
#         # Debugging statements
#         debug_log(f"Index: {idx}, User Index: {user_idx}, Session Index: {session_idx}, Sequence Index: {seq_idx}")
        
#         # Ensure that indices are within valid range
#         if user_idx < len(self.eval_data) and session_idx < len(self.eval_data[user_idx]) and seq_idx < len(self.eval_data[user_idx][session_idx]):
            
#             debug_log(f"The length {len(self.eval_data[user_idx][session_idx][seq_idx])} ")
#             debug_log(f"test ,{user_idx}, {session_idx},{ seq_idx}")
#             data = self.eval_data[user_idx][session_idx][seq_idx]

#             # Debugging statement to check the returned data
#             if data is None:
#                 error_log(f"Returned data is None for index: {idx} in testdata")

#             return data,user_idx
#         else:
#             pass


In [ ]:
# The lenth of sequence calculate dynamically
class TestDataset(Dataset):
    def __init__(self, eval_data):
        self.eval_data = eval_data
        self.num_sessions = [len(user_sessions) for user_sessions in self.eval_data]  # List of number of sessions for each user
        self.num_seqs = [len(session) for user_sessions in self.eval_data for session in user_sessions]  # Total sequences across all users

    def __len__(self):
        # Total length of dataset will be the sum of all sequences across all users and sessions
        return sum(len(self.eval_data[user_idx][session_idx]) for user_idx in range(len(self.eval_data))
                   for session_idx in range(len(self.eval_data[user_idx])))

    def __getitem__(self, idx):
        # Find the user index and session index dynamically
        cumulative_length = 0
        for user_idx in range(len(self.eval_data)):
            for session_idx in range(len(self.eval_data[user_idx])):
                session_length = len(self.eval_data[user_idx][session_idx])
                if cumulative_length + session_length > idx:
                    seq_idx = idx - cumulative_length
                    data = self.eval_data[user_idx][session_idx][seq_idx]

                    # Debugging statements
                    debug_log(f"Index: {idx}, User Index: {user_idx}, Session Index: {session_idx}, Sequence Index: {seq_idx}")

                    # Check if data is None
                    if data is None:
                        error_log(f"Returned data is None for index: {idx} in testdata")
                    return data, user_idx

                cumulative_length += session_length
        
        # If we get here, idx is out of bounds
        raise IndexError("Index out of bounds for dataset.")

In [ ]:
best_accuracy = BEST_ACCURACY
best_loss = BEST_LOSS
training_losses = []
validation_losses = []
feature_embeddings_train=[]
saved_models = []
max_saved_models = MAX_SAVED_MODELS  

# Training phase and store the model using both losses and Accuracy, Store Training accuracy

The number of model store based on variable MAX_SAVED_MODEL

In [ ]:
def find_worst_loss_and_acc_in_existing_model(saved_models):
    worst_loss = float('-inf')
    worst_accuracy = float('inf')
    for model_info in saved_models:
        loss = model_info['loss']
        accuracy = model_info['accuracy']
        if worst_loss <  loss:
            worst_loss = loss
        if worst_accuracy > accuracy:
            worst_accuracy = accuracy
    return  worst_loss, worst_accuracy

def memory_is_low():
    """Check if memory is low. You can implement your own logic here."""
    # Placeholder logic; replace with actual memory checking
    import psutil
    return psutil.virtual_memory().available < (100 * 1024 * 1024)  # Less than 100 MB

def find_worst_model_by_accuracy(saved_models):
    worst_model = None
    worst_accuracy = float('inf')
    for model_info in saved_models:
        if model_info['accuracy'] < worst_accuracy:
            worst_accuracy = model_info['accuracy']
            worst_model = model_info
    if worst_model:
        print_log(f"Worst model - Path: {worst_model['path']}, Accuracy: {worst_model['accuracy']:.6f}, Loss: {worst_model['loss']:.6f}")
    if worst_model:
        if os.path.exists(worst_model['path']):
            os.remove(worst_model['path'])
            print_log(f"Deleted worst model by less accuracy: {worst_model['path']} with Accuracy: {worst_model['accuracy']:.6f}, Loss: {worst_model['loss']:.6f}")
        saved_models.remove(worst_model)

def find_worst_model_by_loss(saved_models):
    worst_model = None
    worst_loss = float('-inf')
    for model_info in saved_models:
        if model_info['loss'] > worst_loss:
            worst_loss = model_info['loss']
            worst_model = model_info
    if worst_model:
        print_log(f"Worst model - Path: {worst_model['path']}, Accuracy: {worst_model['accuracy']:.6f}, Loss: {worst_model['loss']:.6f}")
    if worst_model:
        if os.path.exists(worst_model['path']):
            os.remove(worst_model['path'])
            print_log(f"Deleted worst model by higher loss: {worst_model['path']} with Accuracy: {worst_model['accuracy']:.6f}, Loss: {worst_model['loss']:.6f}")
        saved_models.remove(worst_model)

def find_worst_model_by_combined_metric(saved_models, weight_loss=0.5, weight_accuracy=0.5):
    worst_model = None
    worst_combined_metric = float('inf')
    combined_matrix_values = []
    max_loss = 100
    min_loss = 0
    max_accuracy = 100
    min_accuracy = 0
    for model_info in saved_models:
        normalized_loss = (model_info['loss'] - min_loss) / (max_loss - min_loss)
        normalized_accuracy = (model_info['accuracy'] - min_accuracy) / (max_accuracy - min_accuracy)
        # find combined metric, We want high accuracy and less losses
        combined_metric = (weight_accuracy * normalized_accuracy) -(weight_loss * normalized_loss)
        combined_matrix_values.append(combined_metric)
        model_info["combined_metric"] = combined_metric
        
        # Find the model with the smallest combined metric
        if combined_metric < worst_combined_metric:
            worst_combined_metric = combined_metric
            worst_model = model_info
    if worst_model:
        print_log(f"Worst model : Path: {worst_model['path']}, Accuracy: {worst_model['accuracy']:.6f}, Loss: {worst_model['loss']:.6f}")
        print_log(f"Because this is have minimum combined matrix = {worst_combined_metric}, All combined matric {combined_matrix_values}")
    if worst_model:
        if os.path.exists(worst_model['path']):
            os.remove(worst_model['path'])
            print_log(f"Deleted worst model by less accuracy and higher loss: {worst_model['path']} with Accuracy: {worst_model['accuracy']:.6f}, Loss: {worst_model['loss']:.6f}")
        saved_models.remove(worst_model)

total_losses = []
total_accuracy = []

for i in range(init_epoch, epochs):
# for epoch in range(epochs):
    model_saved = 0
    print(f"Epoch {i+1} started")
    t_loss = 0.0
    start = time.time()
    model.train(True)
    
    for batch_idx, item in enumerate(dataloader):
        anchor, positive, negatives, anchor_class, negative_classes  = item
        optimizer.zero_grad()

        # Forward pass for anchor, positive, and negatives
        anchor_scores, anchor_features = model(anchor.float())
        positive_scores, positive_features = model(positive.float())

        # Separate negative class scores and features
        negative_class_scores, negative_features = zip(*[model(neg.float()) for neg in negatives])
        negative_class_scores = []
        negative_features = []
        for neg in negatives:
            negative_class_score, negative_feature = model(neg.float())
            negative_class_scores.append(negative_class_score)
            negative_features.append(negative_feature)
        
        # Convert negative features to a tensor
        negative_features = torch.stack(negative_features)  # Shape: (batch_size, num_negatives, embedding_dim)
        
        # Debug prints to check sizes
#         print(f"Anchor features shape: {anchor_features.shape}")
#         print(f"Positive features shape: {positive_features.shape}")
#         print(f"Negative features shape: {negative_features.shape}")

#         # Check if dimensions are correct
#         assert anchor_features.size(0) == negative_features.size(0), "Batch sizes must match"

        # Compute N-Pair Loss with negative features
        nPair_loss = loss_fn(anchor_features, positive_features, negative_features)

        # Classification with CrossEntropy Loss
        all_class_scores = torch.cat([anchor_scores, positive_scores] + negative_class_scores, dim=0)
        all_labels = torch.cat([anchor_class, anchor_class] + negative_classes, dim=0)
        class_loss = nn.CrossEntropyLoss()(all_class_scores, all_labels)

        # Combine the losses
        total_loss = WEIGHT_NPAIR_LOSS * nPair_loss + WEIGHT_CLASSIFIER_LOSS * class_loss
        total_loss.backward()
        optimizer.step()
        
        
        t_loss += total_loss.item()
        
    
    t_loss /= len(dataloader)
    training_losses.append(t_loss)
    
#     accuracy_train = accuracy_score(all_labels_train, predicted_classes_train)
#     print("accuracy of train", accuracy_train)

    
    # Validation phase
    model.eval()
    v_loss = 0.0
    all_preds = []
    all_labels = []
    t_dataset = TestDataset(validation_data)
    
    t_dataloader = DataLoader(t_dataset, batch_size=batch_size, shuffle=False)
    tot = 0
    print_log(f"The lenth of t_dataloader : {len(t_dataloader)}")
    for batch_idx_t, item_t in enumerate(t_dataloader):
        with torch.no_grad():
            val = tot // 992
            tot += 1

            item_t_in, class_label = item_t
            # Get model outputs
            logits = model(item_t_in.float()) 
            
            # Apply softmax to get probabilities
            probabilities = torch.softmax(logits[0], dim=1)  # Assuming logits[0] contains class scores
            predicted_classes = torch.argmax(probabilities, dim=1)  # Get predicted classes
            true_labels = class_label
            correct_predictions = (predicted_classes == true_labels)
            
            accuracy = correct_predictions.sum().item() / len(true_labels)
            
#             print(f"Batch {batch_idx_t} - True Labels: {true_labels.tolist()} - Predicted: {predicted_classes.tolist()} -Class Labels: {class_label}")
            all_preds.extend(predicted_classes.tolist())
            all_labels.extend(true_labels.tolist())
#             val_loss = classification_loss_fn(class_scores, true_labels)
#             v_loss += val_loss.item()
#     v_loss /= len(t_dataloader)
#     validation_losses.append(v_loss)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    print_log(f"loop done  {tot}")
    end = time.time()
    
    total_losses.append(t_loss)
    total_accuracy.append(accuracy)
    
    print_log(f"------> Epoch No: {i+1} => Loss: {t_loss:.6f} >> Accuracy: {accuracy:.6f} >> Precision: {precision:.6f} >> Recall: {recall:.6f} >> F1: {f1:.6f} >> Time: {end-start:.2f}")
#     scheduler.step(t_loss)
    # log metrics to wandb
    wandb.log({"accuracy": accuracy, "loss": t_loss, "precision" : precision, "recall" : recall, "f1" : f1})

    
    # Check memory status
    if memory_is_low():
        print_log(f"Memory is running low, attempting to free up space by deleting the worst model.")
        # find_worst_model_by_loss(saved_models)
        # find_worst_model_by_accuracy(saved_models)
        find_worst_model_by_combined_metric(saved_models)
        # assign last best loss and accuracy in existing models
        if len(saved_models) > max_saved_models:
            best_loss, best_accuracy = find_worst_loss_and_acc_in_existing_model(saved_models)

    # Save model if validation loss improves and This model not saved before
    if t_loss < best_loss:
        print_log(f"Loss improved from {best_loss:.6f} to {t_loss:.6f}. ")
        if model_saved:
            print_log("Already model saved. Skip this model by loss.")
        else:
            print_log("Saving model.................")
            model_path = f"{best_model_save_path}/epoch_{i+1}_accuracy_{accuracy:.6f}_loss_{t_loss:.6f}.pt"
            torch.save(model, model_path)
            print_log(f"Best model saved at (by loss): {model_path}")
            saved_models.append({'path': model_path, 'loss': t_loss, 'accuracy' : accuracy})
            # Model saved
            model_saved = 1
    
            if len(saved_models) > max_saved_models:
                # find_worst_model_by_loss(saved_models)
                # find_worst_model_by_accuracy(saved_models)
                find_worst_model_by_combined_metric(saved_models)
            
                # assign last best loss and accuracy in existing models
                best_loss, best_accuracy = find_worst_loss_and_acc_in_existing_model(saved_models)


    # Save model if validation accuracy improves
    if accuracy > best_accuracy :
        print_log(f"Accuracy improved from {best_accuracy:.6f} to {accuracy:.6f}.")
        if model_saved:
            print_log("But, Already model saved. Skip this model by accuracy.")
        else:
            print_log("Saving model.................")
            model_path = f"{best_model_save_path}/epoch_{i+1}_accuracy_{accuracy:.6f}_loss_{t_loss:.6f}.pt"
            torch.save(model, model_path)
            print_log(f"Best model saved at (by accuracy): {model_path}")
            saved_models.append({'path': model_path, 'loss' : t_loss, 'accuracy': -accuracy})
            # Model saved
            model_saved = 1
    
            if len(saved_models) > max_saved_models:
                # find_worst_model_by_loss(saved_models)
                # find_worst_model_by_accuracy(saved_models)
                find_worst_model_by_combined_metric(saved_models)
            
                # assign last best loss and accuracy in existing models
                best_loss, best_accuracy = find_worst_loss_and_acc_in_existing_model(saved_models)
    

print(f"total_losses : {total_losses}")
print(f"total_accuracy : {total_accuracy}")

# Get best model

In [ ]:
directory = '/kaggle/working/best_models'

# Get the list of files in the directory
files = os.listdir(directory)

# Print the list of files
print_log(f"The model file directory : {directory}")
for file in files:
    print_log(f"file  :  {file}")

# # Function to extract EER from filename
# def extract_accuracy(filename):
#     try:
#         parts = filename.split('_')
#         accuracy_index = parts.index('accuracy') + 1
#         accuracy_value = float(parts[accuracy_index].replace('.pt', ''))
#         return accuracy_value
#     except (ValueError, IndexError):
#         return float('inf')

# # Find the file with the lowest EER
# best_model_file = max(files, key=extract_accuracy)
# best_acc = extract_accuracy(best_model_file)

# Function to extract accuracy and loss from filename
def extract_info(filename):
    parts = filename.split('_')
    epoch_value = int(parts[parts.index('epoch') + 1])
    accuracy_value = float(parts[parts.index('accuracy') + 1])
    loss_value = float(parts[parts.index('loss') + 1].replace('.pt', ''))
    return epoch_value, accuracy_value, loss_value


# Function to find the best model using both loss and accuracy
def find_best_model_using_normalization(file_list, weight_loss=0.5, weight_accuracy=0.5):
    best_model = None
    best_combined_metric = float('-inf')
    combined_matrix_values = []
    saved_models = []

    # Extract accuracy and loss from the filenames
    for f in file_list:
        file_info =  extract_info(f)
        saved_models.append({'filename': f,'epoch' : file_info[0], 'accuracy': file_info[1], 'loss': file_info[2]})

    # Calculate statistical parameters needed for normalization
    losses = [model['loss'] for model in saved_models]
    accuracies = [model['accuracy'] for model in saved_models]

    max_loss = 100
    min_loss = 0
    max_accuracy = 100
    min_accuracy = 0

    # Apply the selected normalization method
    for model_info in saved_models:
        loss = model_info['loss']
        accuracy = model_info['accuracy']
        normalized_loss = (loss - min_loss) / (max_loss - min_loss)
        normalized_accuracy = (accuracy - min_accuracy)/(max_accuracy - min_accuracy)
        
        # Combined metric: prioritize higher accuracy and lower loss
        combined_metric = (weight_accuracy * normalized_accuracy) - (weight_loss * normalized_loss)
        combined_matrix_values.append(combined_metric)

        # Find the model with the highest combined metric
        if combined_metric > best_combined_metric:
            best_combined_metric = combined_metric
            best_model = model_info

    # Print best model information
    if best_model:
        print_log(f"Best model - File: {best_model['filename']}, Accuracy: {best_model['accuracy']:.6f}, Loss: {best_model['loss']:.6f}")
        print_log(f"Best combined metric: {best_combined_metric} in all combined metrics: {combined_matrix_values}")

    return best_model

# Find the best model using Min-Max Normalization
print_log("Best Model using Min-Max Normalization")
best_model = find_best_model_using_normalization(files)



best_model_file = best_model['filename']
best_acc = best_model['accuracy']
best_loss = best_model['loss']
wandb.log({
    "best_model_epoch": best_model["epoch"],
    "best_model_accuracy": best_model['accuracy'],
    "best_model_loss": best_model['loss']
})

# Load the best model
best_model_path = os.path.join(directory, best_model_file)
test_model = torch.load(best_model_path)

print_log(f"Best model: {best_model_file} with accuracy: {best_acc} and loss: {best_loss}")
print_log(f"Loaded model from: {best_model_path}")



In [ ]:
# Plotting training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(training_losses, label='Training Loss')
plt.plot(validation_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.show()
print_log("Training loss over epoches polt is create sucessfully")

In [ ]:
test_dataset = TestDataset(test_data)
test_dataloader = DataLoader(test_dataset, batch_size=2)
feature_embeddings = []
tot=0
all_preds_test = []
all_labels_test = []

In [ ]:
tot=0
for batch_idx_t, item_t in enumerate(test_dataloader):
    with torch.no_grad():
        val=tot//162 ## this value needs to be changed based on the sequence from each activity
        tot+=1
        item_t_in,class_label=item_t
#         print("tot",tot)
        true_labels=class_label

        item_out = test_model(item_t_in.float())
        class_scores=item_out[0]
#         print(class_scores)
        feature_embeddings.append(item_out[1])
        predicted_classes = torch.argmax(class_scores, dim=1)
        all_preds_test.extend(predicted_classes.tolist())
        all_labels_test.extend(true_labels.tolist())

In [ ]:
# Move each tensor to CPU, detach from computation graph, and convert to NumPy array
feature_embeddings_cpu = [emb.cpu().detach().numpy() for emb in feature_embeddings]

# Check for NaNs in individual arrays
for i, emb_np in enumerate(feature_embeddings_cpu):
    if np.isnan(emb_np).any():
        print(f"NaN detected in feature_embeddings_cpu at index {i}")

# Concatenate the list of numpy arrays into a single numpy array
feature_embeddings_np = np.concatenate(feature_embeddings_cpu, axis=0)

# Check for NaNs in the concatenated array
if np.isnan(feature_embeddings_np).any():
    print("NaN detected in feature_embeddings_np")

# Perform PCA to reduce dimensions to 2
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(feature_embeddings_np)

# Check for NaNs in PCA results
if np.isnan(principal_components).any():
    print("NaN detected in principal_components")

# Plot PCA results
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(principal_components[:, 0], principal_components[:, 1], alpha=0.5)
plt.title('PCA of Feature Embeddings')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()
print_log("PCA of Feature embeddings polt is create sucessfully")


In [ ]:
# Assuming all_labels_test contains the true labels for the data points
# Map each class to a different color using a colormap
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

# Convert feature embeddings to NumPy array (already done in previous steps)
# feature_embeddings_cpu = [emb.cpu().detach().numpy() for emb in feature_embeddings]
# feature_embeddings_np = np.concatenate(feature_embeddings_cpu, axis=0)

# Perform PCA to reduce dimensions to 2
pca = PCA(n_components=2)
principal_components = pca.fit_transform(feature_embeddings_np)

# Create a scatter plot with different colors for each class
plt.figure(figsize=(8, 6))

# Get a colormap with enough colors for each class
num_classes = len(np.unique(all_labels_test))
palette = sns.color_palette("hsv", num_classes)

# Scatter plot with colors based on class labels
for class_id in np.unique(all_labels_test):
    # Filter the points belonging to the current class
    indices = np.where(np.array(all_labels_test) == class_id)
    plt.scatter(principal_components[indices, 0], 
                principal_components[indices, 1], 
                alpha=0.6, 
                label=activity_id_mapping[class_id], 
                color=palette[class_id])

# Add plot details
plt.title('PCA of Feature Embeddings by Class')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title="Class", loc='best', bbox_to_anchor=(1, 1), ncol=1)  # Show legend outside plot
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()

print_log("PCA of Feature embeddings plot with class colors created successfully")


In [ ]:
accuracy_test = accuracy_score(all_labels_test, all_preds_test)
precision_test = precision_score(all_labels_test, all_preds_test, average='macro')
recall_test = recall_score(all_labels_test, all_preds_test, average='macro')
f1_test = f1_score(all_labels_test, all_preds_test, average='macro')

print_log(f"Accuracy: {accuracy_test:.6f} - Precision: {precision_test:.6f} - Recall: {recall_test:.6f} - F1: {f1_test:.6f}")


In [ ]:
conf_matrix = confusion_matrix(all_labels_test, all_preds_test)

# Calculate the accuracy for each class
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# Print the accuracy for each class
for i, class_accuracy in enumerate(class_accuracies):
    print_log(f"Accuracy for class {i}: {class_accuracy:.6f}")


plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()
print_log("Confusion matrix is create sucessfully using test")
wandb.log({"confusion_matrix": wandb.Image('confusion_matrix.png')})

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Initialize lists to store results
class_names = []
precision_list = []
recall_list = []
f1_list = []

# Calculate precision, recall, and f1-score for each class
for class_id, class_name in activity_id_mapping.items():
    # Precision, recall, and f1 for the current class without averaging
    precision = precision_score(all_labels_test, all_preds_test, labels=[class_id], average=None, zero_division=0)
    recall = recall_score(all_labels_test, all_preds_test, labels=[class_id], average=None, zero_division=0)
    f1 = f1_score(all_labels_test, all_preds_test, labels=[class_id], average=None, zero_division=0)
    
    # Append results to respective lists
    class_names.append(class_name)
    precision_list.append(precision[0])  # Since labels=[class_id] returns a list
    recall_list.append(recall[0])        # Extract the first element for that class
    f1_list.append(f1[0])


class_names = class_names[0: CLASSES]
precision_list = precision_list[0: CLASSES]
recall_list = recall_list[0: CLASSES]
f1_list = f1_list[0: CLASSES]


# Calculate the average across all classes (macro average)
average_precision = sum(precision_list) / len(precision_list)
average_recall = sum(recall_list) / len(recall_list)
average_f1 = sum(f1_list) / len(f1_list)



class_names.append('Average')
recall_list.append(average_recall)
precision_list.append(average_precision)
f1_list.append(average_f1)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Class': class_names,
    'Recall': recall_list,
    'Precision': precision_list,
    'F1-score': f1_list
})

results_df.to_csv('class_precision_recall_f1.csv', index=False)

# Display the final table
results_df


In [ ]:
conf_matrix = confusion_matrix(all_labels, all_preds)

# Calculate the accuracy for each class
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# Print the accuracy for each class
for i, class_accuracy in enumerate(class_accuracies):
    print_log(f"Accuracy for class {i}: {class_accuracy:.6f}")

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix_validation.png')
plt.show()
print_log("Validation Confusion matrix is create sucessfully")

In [ ]:
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()